In [8]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [20]:
#from the MNIST main dataset create training and testing datasets and convert to tensors using ToTensor
train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)
#Load the data
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

#creating the neural network
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        #Flatten is to flatten the Tensor into a 1D tensor
        self.flatten = nn.Flatten()
        #Define the Linear and ReLU layer stack(Basically stacking different layers that do computation one on top of another)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    #The forward function does the actual computation itself
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

#store the entire neural network in model
model = NeuralNetwork()

In [17]:
#define hyperparameters(These help control the optimization process )
learning_rate = 1e-3
batch_size = 64
epochs = 10
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [18]:
#create training and testing data
def train_loop(dataloader, model, loss_func, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (x, y) in enumerate(dataloader):
        #predicted value obtained from model is stored in pred
        pred = model(x)
        #loss value calulated and stored in loss
        loss = loss_func(pred, y)
        #Call the backward function so that we can get the gradients
        loss.backward()
        #optimizer.step performs a single optimization step
        optimizer.step()
        #sets all gradient values of the optimized tensor to zero (this is just to save memory as calling backward function stores the gradient values)
        optimizer.zero_grad()

        if batch % 100 == 0:
            #obtain loss and current iterate
            loss, current = loss.item(), (batch + 1) * len(x)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_func):
    #evaluate model
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    #torch.nograd allows us to perform functions without using gradient function
    with torch.no_grad():
        for x, y in dataloader:
            pred = model(x)
            #keep adding the total losses
            test_loss += loss_func(pred, y).item()
            #correct stores how many are correct outputs has been done which is used to find accuracy
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Error: \n Accuracy: {(100*correct):>0.1f}%, avg loss: {test_loss:>8f} \n")

In [19]:
#print the losses and acuracy
for i in range(epochs):
    print(f"Epoch: {i + 1}\n-----------------------------------")
    train_loop(train_dataloader, model, loss_func, optimizer)
    test_loop(test_dataloader, model, loss_func)

Epoch: 1
-----------------------------------
loss: 2.304921 [   64/60000]


/home/kyo/.local/lib/python3.11/site-packages/torch/autograd/__init__.py:200: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


loss: 2.292873 [ 6464/60000]
loss: 2.274122 [12864/60000]
loss: 2.265943 [19264/60000]
loss: 2.247003 [25664/60000]
loss: 2.214440 [32064/60000]
loss: 2.217565 [38464/60000]
loss: 2.185335 [44864/60000]
loss: 2.186031 [51264/60000]
loss: 2.142268 [57664/60000]
Error: 
 Accuracy: 50.2%, avg loss: 2.144970 

Epoch: 2
-----------------------------------
loss: 2.159708 [   64/60000]
loss: 2.150304 [ 6464/60000]
loss: 2.092492 [12864/60000]
loss: 2.105828 [19264/60000]
loss: 2.047624 [25664/60000]
loss: 1.992519 [32064/60000]
loss: 2.008750 [38464/60000]
loss: 1.932858 [44864/60000]
loss: 1.939714 [51264/60000]
loss: 1.856231 [57664/60000]
Error: 
 Accuracy: 58.9%, avg loss: 1.858107 

Epoch: 3
-----------------------------------
loss: 1.898138 [   64/60000]
loss: 1.864735 [ 6464/60000]
loss: 1.740834 [12864/60000]
loss: 1.781711 [19264/60000]
loss: 1.661835 [25664/60000]
loss: 1.625343 [32064/60000]
loss: 1.629533 [38464/60000]
loss: 1.535074 [44864/60000]
loss: 1.566079 [51264/60000]
loss